In [1]:
import pandas as pd

df_geo = pd.read_csv("./File/geographical_places_dataset_2.csv")
df_geo.head(1)

,title,gemini_text,gemini_sentences,annotation,gemini_text_samples,annotation_note
0,Hà Nội,"Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21°0...","['Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21...","['0', '0.5', '0', '0.5', '0.5', '0']","['Hà Nội, thủ đô của Việt Nam, nằm ở tọa độ đị...","[None, 'Hà Nội chỉ nằm trên lưu vực Sông Hồng ..."


In [2]:
df_his = pd.read_csv("./File/historical_figures_dataset.csv")
df_his.head(1)

,title,gemini_text,gemini_sentences,annotation,gemini_text_samples,annotation_note
0,Hồ Chí Minh,"Hồ Chí Minh (1890-1969), tên khai sinh là Nguy...","['Hồ Chí Minh (1890-1969), tên khai sinh là Ng...","['0', '0', '0', '0', '0']","['Hồ Chí Minh (1890-1969), tên khai sinh là Ng...","[None, None, None, None, None]"


In [3]:
merged_df = pd.concat([df_geo, df_his], axis=0)
len(merged_df)

20

In [4]:
df_34 = pd.read_csv("./File/Vietnamese_hallucination_34_annotated.csv")
df_34.head(1)

,title,gemini_text,gemini_sentences,annotation,gemini_text_samples
0,Khởi nghĩa Hai Bà Trưng,Khởi nghĩa Hai Bà Trưng là một cuộc khởi nghĩa...,['Khởi nghĩa Hai Bà Trưng là một cuộc khởi ngh...,"['accurate', 'minor_inaccurate', 'accurate', '...",['Khởi nghĩa Hai Bà Trưng là một cuộc khởi ngh...


In [5]:
len(df_his.loc[0, 'annotation'])

25

In [6]:
# add annotation_note column to df_34 with None array have the same length with annotation column
from ast import literal_eval
label_mapping = {
    'accurate': "0",
    'minor_inaccurate': "0.5",
    'major_inaccurate': "1.0",
}
# revert label_mapping
label_mapping = {v: k for k, v in label_mapping.items()}
def safe_literal_eval(val):
    try:
        return literal_eval(val)
    except (ValueError, SyntaxError):
        return val
print(label_mapping)
def prepare(df, replace_label=True):
    df['annotation'] = df['annotation'].apply(safe_literal_eval)
    if 'annotation_note' not in df.columns:
        df['annotation_note'] = df['annotation'].apply(lambda x: [None] * len(x))
    if replace_label:
        df['annotation'] = df['annotation'].apply(lambda x: [label_mapping.get(i) for i in x])
    return df
df_34 = prepare(df_34, replace_label=False)
merged_df = prepare(merged_df)
finnal_df = pd.concat([merged_df, df_34], axis=0)
finnal_df.head()

{'0': 'accurate', '0.5': 'minor_inaccurate', '1.0': 'major_inaccurate'}


,title,gemini_text,gemini_sentences,annotation,gemini_text_samples,annotation_note
0,Hà Nội,"Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21°0...","['Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21...","[accurate, minor_inaccurate, accurate, minor_i...","['Hà Nội, thủ đô của Việt Nam, nằm ở tọa độ đị...","[None, 'Hà Nội chỉ nằm trên lưu vực Sông Hồng ..."
1,Thành phố Hồ Chí Minh,Thành phố Hồ Chí Minh (TP.HCM) nằm ở phía Đông...,['Thành phố Hồ Chí Minh (TP.HCM) nằm ở phía Đô...,"[accurate, accurate, minor_inaccurate, minor_i...",['Thành phố Hồ Chí Minh (TP. HCM) nằm ở phía Đ...,"[None, None, 'Hồ Chí Minh nằm ở hạ lưu sông Sà..."
2,Vịnh Hạ Long,"Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, thu...","['Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, t...","[accurate, minor_inaccurate, accurate, minor_i...","['Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, t...","[None, '20°54′B 107°12′Đ', None, 'phía Tây và ..."
3,Sapa,"Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Việt...","['Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Vi...","[accurate, minor_inaccurate, minor_inaccurate,...","['Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Vi...","[None, '22°20′8″B 103°50′31″Đ', '1.500 m – 1.8..."
4,Đà Nẵng,Đà Nẵng là một thành phố ven biển thuộc miền T...,['Đà Nẵng là một thành phố ven biển thuộc miền...,"[minor_inaccurate, minor_inaccurate, accurate,...",['Đà Nẵng là một thành phố trực thuộc trung ươ...,"['16°04′36″B 108°13′22″Đ', '1.284,88 km2, Quản..."


In [7]:
finnal_df.to_csv("./File/Vietnamese_hallucination_annotated.csv", index=False)